In [1]:
import torch
import pathpyG as pp
from collections import defaultdict

In [2]:
def temporal_shortest_paths(g: pp.TemporalGraph) -> defaultdict:
    """
    Calculates all shortest paths between all pairs of nodes 
    based on a set of empirically observed paths.
    """
    sp = defaultdict(lambda: defaultdict(set))
    sp_lengths = torch.full((g.N, g.N), float('inf'))
    return None